<h1>Proyecto Tecnologías de Búsqueda en la Web</h1>
<h3>Integrantes</h3>
<ul><li>Sebastián Aranda</li><li>Felipe Santander</li></ul>

<h2>Librerías</h2>

In [36]:
import gensim
from gensim import corpora
from gensim.corpora import Dictionary, bleicorpus
from gensim.models import ldamodel
from gensim.models import Phrases

from collections import defaultdict

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from random import randint

import pyLDAvis
import re
import os
import json

from __future__ import division

<h2>Creación del Corpus</h2>

In [37]:
corpus_path = "cyberbullying_corpus/"
roastme_dataset_path = "Corpus_builder_and_pre/Dataset/"
balance_dataset_path = "Corpus_builder_and_pre/Neutros&Positivos/"

In [38]:
#Create bad words list
bad_words = []
with open('badwords','r') as bad_words_file:
    for word in bad_words_file:
        word = word.replace('\n','').decode('unicode_escape').encode('ascii','ignore')
        if word != '':
            bad_words.append(word)
print("Badwords List Size: "+str(len(bad_words)))
        
#Remove keywords in cyberbullying from stopwords list
keywords = ["you", "your", "he", "she", "it"]
stopword_list = [stopword for stopword in stopwords.words('english') if stopword not in keywords]
puncts = ".,:;?!()[]{}~+-\"\'#$%&\/"
digits = "0123456789"

#Create the corpus
print("Creating the corpus...")
mcorpus = dict()
mcorpus['roastme'] = list()
mcorpus['balance'] = list()

print("Adding Roastme Dataset...")
count = 0
for name in os.listdir(roastme_dataset_path):
    if (count >= 6500):
        break
    if name.endswith('.json'):
        with open(roastme_dataset_path+'/'+name) as f:
            op_json = json.loads(f.read())
            try:
                for child in op_json[1]['data']['children']:    
                    #Extrae el comentario
                    comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                    #Elimina los links
                    comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)

                    for sym in puncts:
                        comment_text = comment_text.replace(sym," ")
                    for num in digits:
                        comment_text = comment_text.replace(num," ")

                    tokens_comment = [word for word in comment_text.lower().split() if word not in stopword_list]
                    mcorpus['roastme'].append(tokens_comment)
                    
                    count += 1
                    if (count >= 6500):
                        break
                        
                    try:
                        for child in child['data']['replies']['data']['children']:
                            comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                            comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)
                    
                            for sym in puncts:
                                comment_text = comment_text.replace(sym," ")
                            for num in digits:
                                comment_text = comment_text.replace(num," ")
                    
                            tokens_comment = [word for word in comment_text.lower().split() if word not in stopword_list]
                            mcorpus['roastme'].append(tokens_comment)
                            
                            count += 1
                            if (count >= 6500):
                                break
                            
                    except Exception:
                        pass
            except Exception:
                pass

print("Adding Neutral&Positive Dataset...")
count = 0
for name in os.listdir(balance_dataset_path):
    if (count >= 4500):
        break
    if name.endswith('.json'):
        with open(balance_dataset_path+'/'+name) as f:
            op_json = json.loads(f.read())
            try:
                for child in op_json[1]['data']['children']:    
                    #Extrae el comentario
                    comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                    #Elimina los links
                    comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)

                    for sym in puncts:
                        comment_text = comment_text.replace(sym," ")
                    for num in digits:
                        comment_text = comment_text.replace(num," ")

                    tokens_comment = [word for word in comment_text.lower().split() if word not in stopwords.words('english')]
                    mcorpus['balance'].append(tokens_comment)
                    
                    count += 1
                    if (count >= 4500):
                        break
                
                    try:
                        for child in child['data']['replies']['data']['children']:
                            comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                            comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)
                    
                            for sym in puncts:
                                comment_text = comment_text.replace(sym," ")
                            for num in digits:
                                comment_text = comment_text.replace(num," ")
                    
                            tokens_comment = [word for word in comment_text.lower().split() if word not in stopwords.words('english')]
                            mcorpus['balance'].append(tokens_comment)
                            
                            count += 1
                            if (count >= 4500):
                                break
                            
                    except Exception:
                        pass
            except Exception:
                pass

#Setting Trainning Corpus Cross Fold Validation: 80-20
print("Roastme Dataset Size: "+str(len(mcorpus['roastme']))+" docs")
print("Balance Dataset Size: "+str(len(mcorpus['balance']))+" docs")

roastme_trainning_size = int(len(mcorpus['roastme'])*0.8)
balance_trainning_size = int(len(mcorpus['balance'])*0.8)

train_corpus = mcorpus['roastme'][:roastme_trainning_size] + mcorpus['balance'][:balance_trainning_size]
test_corpus = mcorpus['roastme'][roastme_trainning_size:] + mcorpus['balance'][balance_trainning_size:]
test_corpus_only_bad = mcorpus['roastme'][roastme_trainning_size:]
test_corpus_only_balance = mcorpus['balance'][balance_trainning_size:]

print("Trainning Corpus Size: "+str(len(train_corpus)))
print("Test Corpus Size: "+str(len(test_corpus)))

#Remove words that appear only once
frequency = defaultdict(int)
for doc in train_corpus:
    for token in doc:
        frequency[token] += 1
docs = [[token for token in doc if frequency[token] > 1] for doc in train_corpus]

# Lemmatize all words in documents.
lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            docs[idx].append(token)

#Create and save dictionary
dictionary = corpora.Dictionary(docs)
#dictionary.filter_extremes(no_below=20, no_above=0.8) # Filter out words that occur less than 20 documents, or more than 50% of the documents.

#Convert documents to vectors
corpus = [dictionary.doc2bow(doc) for doc in docs]

#Increasing weight of bad words:
bad_words_ids = [word_id for word_id, word in dictionary.iteritems() if word in bad_words]
ngrams_ids = [word_id for word_id, word in dictionary.iteritems() if "_" in word]

for doc_idx in range(len(corpus)):
    word_id_list = []
    freq_list = []

    for word_id, freq in corpus[doc_idx]:
        word_id_list.append(word_id)
        if (word_id in bad_words_ids or word_id in ngrams_ids):
            freq_list.append(freq*5)
        else:
            freq_list.append(freq)
    
    new_doc = zip(word_id_list,freq_list)
    corpus[doc_idx] = new_doc

print("Saving Corpus and Dictionary...")
dictionary.save('tmp/cyberbullying_dictionary.dict') #Save the dictionary
corpora.BleiCorpus.serialize('tmp/cyberbullying_corpus.lda-c', corpus) #Save the corpus

print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))


Badwords List Size: 376
Creating the corpus...
Adding Roastme Dataset...
Adding Neutral&Positive Dataset...
Roastme Dataset Size: 6501 docs
Balance Dataset Size: 4500 docs
Trainning Corpus Size: 8800
Test Corpus Size: 2201
Saving Corpus and Dictionary...
Number of unique tokens: 4599
Number of documents: 8800


In [ ]:
#Load the dictionary and corpus
if (os.path.exists('tmp/cyberbullying_dictionary.dict') and os.path.exists('tmp/cyberbullying_corpus.lda-c')):
    print('Creating dictionary...')
    %time dictionary = corpora.Dictionary.load('tmp/cyberbullying_dictionary.dict')
    print('Creating corpus...')
    %time corpus = corpora.BleiCorpus('tmp/cyberbullying_corpus.lda-c')
else:
    print("Create the dictionary and corpus first")

In [39]:
#Initialize a model
#print('Creating Tfidf model...')
tfidf = gensim.models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

%time lda_model = ldamodel.LdaModel(corpus, num_topics=2, id2word=dictionary)
lda_model.save('tmp/cyberbullying_ldaModel.lda')
lda_model.print_topics(2,100)

#lda_model_tfidf = ldamodel.LdaModel(corpus_tfidf, num_topics=2, id2word=dictionary)

CPU times: user 23.2 s, sys: 52 ms, total: 23.3 s
Wall time: 23.3 s


[(0,
  u'0.013*"love" + 0.009*"good" + 0.009*"one" + 0.009*"make" + 0.008*"people" + 0.007*"really" + 0.007*"meme" + 0.007*"wholesome_meme" + 0.007*"thank" + 0.007*"great" + 0.007*"day" + 0.007*"know" + 0.007*"nice" + 0.007*"sub" + 0.007*"fuck" + 0.007*"get" + 0.006*"friend" + 0.006*"wholesome" + 0.006*"you" + 0.006*"thanks" + 0.006*"think" + 0.006*"go" + 0.005*"thing" + 0.005*"want" + 0.005*"time" + 0.005*"feel" + 0.005*"fucking" + 0.005*"guy" + 0.005*"_" + 0.005*"your" + 0.005*"happy" + 0.005*"made" + 0.004*"better" + 0.004*"post" + 0.004*"u" + 0.004*"even" + 0.004*"r" + 0.004*"see" + 0.004*"would" + 0.004*"life" + 0.004*"way" + 0.004*"much" + 0.004*"man" + 0.004*"hope" + 0.004*"also" + 0.003*"right" + 0.003*"it" + 0.003*"place" + 0.003*"like" + 0.003*"need" + 0.003*"always" + 0.003*"everyone" + 0.003*"comic" + 0.003*"say" + 0.003*"comment" + 0.003*"still" + 0.003*"could" + 0.003*"best" + 0.003*"actually" + 0.003*"going" + 0.003*"take" + 0.003*"sexy" + 0.003*"op" + 0.003*"positive" +

In [40]:
#Evaluate with roastme dataset (agressive)
corpus = [dictionary.doc2bow(doc) for doc in test_corpus_only_bad]
total_docs = 0
true_positive = 0
false_negative = 0
percent_list = []
for docbow in corpus:
    total_docs += 1
    
    if (len(lda_model[docbow]) == 2):
        if (lda_model[docbow][1][1] > 0.5):
            true_positive += 1
        else:
            false_negative += 1
    else:
        if (lda_model[docbow][0][0] == 1):
            true_positive += 1
        else:
            false_negative += 1

recall = true_positive/(true_positive+false_negative)
print("Recall:")
print(recall)
#print('{0:.16f}'.format(recall))


Recall:
0.71329746349


In [ ]:
#Load LDA Model
lda = models.LdaModel.load('tmp/cyberbullying_ldaModel.lda')